In [1]:
pip install dash-leaflet


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import dash_leaflet as dl

c:\Users\Balkis Mansour\anaconda3\envs\monenv\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [12]:
import pandas as pd

# Nouveau chemin du fichier Excel synchronisé depuis SharePoint/Teams
df = pd.read_excel(r"C:\Users\Balkis Mansour\OLECIO\Plateforme - Olecio.fr\Fichier contacts.xlsx")

# Affiche les premières lignes pour vérification
print(df.head())

                                      Organisation\n        Nom   Prénom  \
0                           100 CHANCES, 100 EMPLOIS   MOURIAUX   Emilie   
1                            Actions Emploi réfugiés        NaN      NaN   
2       APELS (Agence pour l'éducation par le sport)  TRUTTMANN  Quentin   
3                                          Article 1   PHILIPON    Lilas   
4  Association française des managers de la diver...     HAGEGE     Maya   

                                            Fonction  \
0                           Responsable partenariats   
1                                                NaN   
2  Responsable national des territoires d'inclusi...   
3                         Coordinatrices pédagogique   
4                                  Déléguée générale   

                            Email 1                           Email 2  \
0           emilie.cardona@suez.com  gilles.vermotdesroches@lecese.fr   
1  contact@actionemploirefugies.com                         

In [13]:
region_coords = {
    "Auvergne-Rhône-Alpes": [45.5, 4.8],
    "Bourgogne-Franche-Comté": [47.0, 5.0],
    "Bretagne": [48.1, -2.7],
    "Centre-Val de Loire": [47.6, 1.9],
    "Corse": [42.0, 9.0],
    "Grand Est": [48.6, 6.0],
    "Hauts-de-France": [50.3, 3.0],
    "Ile-de-France": [48.8, 2.3],
    "Normandie": [49.0, 0.2],
    "Nouvelle-Aquitaine": [45.7, 0.2],
    "Occitanie": [43.6, 2.5],
    "Pays de la Loire": [47.5, -0.5],
    "Provence-Alpes-Côte d'Azur": [43.8, 6.2],
    "DROM-COM": [15.0, -61.0]  # exemple pour Guadeloupe/Martinique
}


In [14]:
# Liste des colonnes territoires à vérifier dans ton fichier
territoires = list(region_coords.keys())

In [16]:
def generate_markers(data):
    import math
    markers = []
    base_radius = 0.08  # rayon de départ (~8 km)
    step_radius = 0.03  # incrément pour chaque “anneau”

    region_counters = {region: 0 for region in territoires}

    for i, row in data.iterrows():
        for territoire in territoires:
            val = row.get(territoire)
            if isinstance(val, str) and val.strip().upper() == "X":
                base_lat, base_lon = region_coords[territoire]

                # Calcul du placement en cercle
                count = region_counters[territoire]
                angle_deg = (count * 30) % 360
                ring = count // 8 # 12 marqueurs max par cercle
                angle_rad = math.radians(angle_deg)
                radius = base_radius + step_radius * ring

                lat_offset = radius * math.cos(angle_rad)
                lon_offset = radius * math.sin(angle_rad)
                coord = [base_lat + lat_offset, base_lon + lon_offset]

                tooltip_text = f"{row['Organisation']} - {territoire}"
                markers.append(
                    dl.Marker(
                        position=coord,
                        children=dl.Tooltip(tooltip_text),
                        id={'type': 'marker', 'index': f"{i}_{territoire}"}
                    )
                )

                region_counters[territoire] += 1
    return markers



# 🚀 App Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Cartographie des partenaires Olecio"),
    
    html.Div([
        dcc.Input(id="recherche", type="text", placeholder="Recherche par nom ou mot-clé", style={"width": "40%"}),
        html.Button("🔄 Mettre à jour", id="maj-button", n_clicks=0, style={"marginLeft": "10px"})
    ], style={"margin": "20px"}),

    dl.Map([
        dl.TileLayer(),
        dl.LayerGroup(id="markers")
    ], center=[46.5, 2.5], zoom=5, style={'width': '100%', 'height': '70vh'})
])

# 🎯 Callback de mise à jour
from dash.dependencies import Input, Output, State  # ajoute State ici

@app.callback(
    Output("markers", "children"),
    Input("maj-button", "n_clicks"),
    State("recherche", "value")
)
def update_markers(n_clicks, search_val):
    df = pd.read_excel(r"C:\Users\Balkis Mansour\OLECIO\Plateforme - Olecio.fr\Fichier contacts.xlsx")
    df.columns.values[0] = "Organisation"

    # Filtrage selon la recherche
    if search_val:
        df = df[df["Organisation"].str.contains(search_val, case=False, na=False)]

    # Filtrage : au moins un "X" dans les territoires
    df_filtered = df[df[territoires].applymap(lambda x: str(x).strip().upper() == 'X').any(axis=1)]

    return generate_markers(df_filtered)
# ▶️ Lancer l'application
if __name__ == '__main__':
    app.run(debug=True)

C:\Users\Balkis Mansour\AppData\Local\Temp\ipykernel_17196\3715231160.py:74: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



In [9]:
import plotly.express as px
import pandas as pd

# Exemple de données : à remplacer par les tiennes si besoin
data = pd.DataFrame({
    'Ville': ['Paris', 'Lyon', 'Marseille'],
    'Latitude': [48.8566, 45.75, 43.2965],
    'Longitude': [2.3522, 4.85, 5.3698],
    'Nombre de salariés': [1200, 800, 600]
})

# Création de la carte avec Plotly Express
fig = px.scatter_mapbox(
    data,
    lat="Latitude",
    lon="Longitude",
    hover_name="Ville",
    size="Nombre de salariés",
    size_max=30,
    zoom=5,
    mapbox_style="open-street-map"  # Pas besoin de clé API
)

# Export en HTML autonome
fig.write_html("carte_olecio2.html")

print("✅ Carte exportée : 'carte_olecio2.html' est prête et interactive !")





C:\Users\Balkis Mansour\AppData\Local\Temp\ipykernel_17196\666670386.py:13: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



✅ Carte exportée : 'carte_olecio2.html' est prête et interactive !


In [10]:
import webbrowser
webbrowser.open('carte_olecio2.html')


True

In [ ]:
import dash
from dash import html, dcc
import dash_leaflet as dl
import pandas as pd
import math
from dash.dependencies import Input, Output, State

# 📍 Coordonnées des centres de régions
region_coords = {
    "Auvergne-Rhône-Alpes": [45.5, 4.8],
    "Bourgogne-Franche-Comté": [47.0, 5.0],
    "Bretagne": [48.1, -2.7],
    "Centre-Val de Loire": [47.6, 1.9],
    "Corse": [42.0, 9.0],
    "Grand Est": [48.6, 6.0],
    "Hauts-de-France": [50.3, 3.0],
    "Ile-de-France": [48.8, 2.3],
    "Normandie": [49.0, 0.2],
    "Nouvelle-Aquitaine": [45.7, 0.2],
    "Occitanie": [43.6, 2.5],
    "Pays de la Loire": [47.5, -0.5],
    "Provence-Alpes-Côte d'Azur": [43.8, 6.2],
    "DROM-COM": [15.0, -61.0]
}

# 🧭 Liste dynamique des territoires
territoires = list(region_coords.keys())

# 📌 Génération des marqueurs avec placement circulaire
def generate_markers(data):
    markers = []
    base_radius = 0.08
    step_radius = 0.03
    markers_per_ring = 8

    region_counters = {region: 0 for region in territoires}

    for i, row in data.iterrows():
        for territoire in territoires:
            val = row.get(territoire)
            if isinstance(val, str) and val.strip().upper() == "X":
                base_lat, base_lon = region_coords[territoire]
                count = region_counters[territoire]
                angle_deg = (count * (360 / markers_per_ring)) % 360
                ring = count // markers_per_ring
                angle_rad = math.radians(angle_deg)
                radius = base_radius + step_radius * ring

                lat_offset = radius * math.cos(angle_rad)
                lon_offset = radius * math.sin(angle_rad)
                coord = [base_lat + lat_offset, base_lon + lon_offset]

                tooltip_text = f"{row['Organisation']} - {territoire}"
                markers.append(
                    dl.Marker(
                        position=coord,
                        children=dl.Tooltip(tooltip_text),
                        id={'type': 'marker', 'index': f"{i}_{territoire}"}
                    )
                )

                region_counters[territoire] += 1
    return markers

# 🚀 App Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Cartographie des partenaires Olecio"),

    html.Div([
        dcc.Input(id="recherche", type="text", placeholder="Recherche par nom", style={"width": "30%", "marginRight": "20px"}),

        dcc.Dropdown(
            id="filtre-region",
            options=[{"label": r, "value": r} for r in territoires],
            placeholder="Filtrer par région",
            style={"width": "30%", "marginRight": "20px"}
        ),

        html.Button("🔄 Mettre à jour", id="maj-button", n_clicks=0),
    ], style={"margin": "20px", "display": "flex", "flexWrap": "wrap", "gap": "10px"}),

    dl.Map([
        dl.TileLayer(),
        dl.LayerGroup(id="markers")
    ], id="map", center=[46.5, 2.5], zoom=5, style={'width': '100%', 'height': '70vh'})
])

# 🎯 Callback pour mise à jour des marqueurs et zoom sur la région
@app.callback(
    Output("markers", "children"),
    Output("map", "center"),
    Output("map", "zoom"),
    Input("maj-button", "n_clicks"),
    State("recherche", "value"),
    State("filtre-region", "value")
)
def update_markers(n_clicks, search_val, region_val):
    df = pd.read_excel(r"C:\Users\Balkis Mansour\OLECIO\Plateforme - Olecio.fr\Fichier contacts.xlsx")
    df.columns.values[0] = "Organisation"

    # Recherche par nom
    if search_val:
        df = df[df["Organisation"].str.contains(search_val, case=False, na=False)]

    # Filtrage par région
    if region_val:
        df = df[df[region_val].apply(lambda x: str(x).strip().upper() == 'X')]

    # Filtrer les lignes avec au moins un "X"
    df_filtered = df[df[territoires].applymap(lambda x: str(x).strip().upper() == 'X').any(axis=1)]

    # Générer les marqueurs
    markers = generate_markers(df_filtered)

    # Définir le centre de la carte
    if region_val and region_val in region_coords:
        center = region_coords[region_val]
        zoom = 7
    else:
        center = [46.5, 2.5]
        zoom = 5

    return markers, center, zoom

# ▶️ Lancement de l'app
if __name__ == '__main__':
    app.run(debug=True)


C:\Users\Balkis Mansour\AppData\Local\Temp\ipykernel_17196\1829116162.py:112: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

C:\Users\Balkis Mansour\AppData\Local\Temp\ipykernel_17196\1829116162.py:112: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

C:\Users\Balkis Mansour\AppData\Local\Temp\ipykernel_17196\1829116162.py:112: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

C:\Users\Balkis Mansour\AppData\Local\Temp\ipykernel_17196\1829116162.py:112: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

C:\Users\Balkis Mansour\AppData\Local\Temp\ipykernel_17196\1829116162.py:112: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

C:\Users\Balkis Mansour\AppData\Local\Temp\ipykernel_17196\1829116162.py:112: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

C:\Users\Balkis Mansour\AppD

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[20], line 112, in update_markers(
    n_clicks=18,
    search_val='100 chances ,100 emplois',
    region_val='Auvergne-Rhône-Alpes'
)
    109     df = df[df[region_val].apply(lambda x: str(x).strip().upper() == 'X')]
    111 # Filtrer les lignes avec au moins un "X"
--> 112 df_filtered = df[df[territoires].applymap(lambda x: str(x).strip().upper() == 'X').any(axis=1)]
        df = Empty DataFrame
Columns: []
Index: []
        territoires = ['Auvergne-Rhône-Alpes', 'Bourgogne-Franche-Comté', 'Bretagne', 'Centre-Val de Loire', 'Corse', 'Grand Est', 'Hauts-de-France', 'Ile-de-France', 'Normandie', 'Nouvelle-Aquitaine', 'Occitanie', 'Pays de la Loire', "Provence-Alpes-Côte d'Azur", 'DROM-COM']
    114 # Générer les marqueurs
    115 markers = generate_markers(df_filtered)

File c:\Users\Balkis Mansour\anaconda3\envs\monenv\lib\site-

C:\Users\Balkis Mansour\AppData\Local\Temp\ipykernel_17196\1829116162.py:112: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

C:\Users\Balkis Mansour\AppData\Local\Temp\ipykernel_17196\1829116162.py:112: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

C:\Users\Balkis Mansour\AppData\Local\Temp\ipykernel_17196\1829116162.py:112: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

C:\Users\Balkis Mansour\AppData\Local\Temp\ipykernel_17196\1829116162.py:112: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

C:\Users\Balkis Mansour\AppData\Local\Temp\ipykernel_17196\1829116162.py:112: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

C:\Users\Balkis Mansour\AppData\Local\Temp\ipykernel_17196\1829116162.py:112: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

C:\Users\Balkis Mansour\AppD